 [View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/pytorch/pytorch_save_and_load_onnx.ipynb)

 # Save and load onnx model

This example illustrates how to save and load a model accelerated by onnxruntime.
 In this example, we use a ResNet18 model pretrained. Then, by calling `trace(model, accelerator="onnxruntime"...)`, we can obtain a model accelarated by onnxruntime method provided by BigDL-Nano for inference. By calling `save(model_name, path)` , we could save the model to a folder. By calling `load(path)`, we could load the model from a folder.

 To inference using Bigdl-nano InferenceOptimizer, the following packages need to be installed first. We recommend you to use [Miniconda](https://docs.conda.io/en/latest/miniconda.html) to prepare the environment and install the following packages in a conda environment.

 You can create a conda environment by executing:

 ```
 # "nano" is conda environment name, you can use any name you like.
 conda create -n nano python=3.7 setuptools=58.0.4
 conda activate nano
 ```

> 📝 **Note**:
>
> during your installation, there may be some warnings or errors about version, just ignore them.


In [ ]:
# Necessary packages for inference accelaration
!pip install --pre --upgrade bigdl-nano[pytorch]
!pip install onnx onnxruntime onnxruntime

 First, prepare model. We use a pretrained ResNet18 model(`model_ft` in following code) in this example.

In [1]:
import torch
from torchvision.models import resnet18

model_ft = resnet18(pretrained=True)
model_ft.eval()

/home/cpx/anaconda3/envs/junwang-resnext-oob/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/cpx/anaconda3/envs/junwang-resnext-oob/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/home/cpx/anaconda3/envs/junwang-resnext-oob/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

 Accelerated Inference Using ONNX Runtime

In [2]:
from bigdl.nano.pytorch import InferenceOptimizer
ort_model = InferenceOptimizer.trace(model_ft,
                                        accelerator="onnxruntime",
                                        input_sample=torch.rand(1, 3, 224, 224))

x = torch.rand(2, 3, 224, 224)
y_hat = ort_model(x)
predictions = y_hat.argmax(dim=1)
print(predictions)

tensor([111, 111])


 Save Optimized Model
 The saved model files will be saved at "./optimized_model_ort" directory
 There are 2 files in optimized_model_ort, users only need to take ".onnx" file for further usage:
  * nano_model_meta.yml: meta information of the saved model checkpoint
  * onnx_saved_model.onnx: model checkpoint for general use, describes model structure

In [3]:
InferenceOptimizer.save(ort_model, "./optimized_model_ort")

 Load the Optimized Model

In [4]:
loaded_model = InferenceOptimizer.load("./optimized_model_ort")

 Inference with the Loaded Model

In [5]:
y_hat = loaded_model(x)
predictions = y_hat.argmax(dim=1)
print(predictions)


tensor([111, 111])


 > 📚 **Related Readings**
 >
 > - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/nano.html#install)
 > - [How to install BigDL-Nano in Google Colab](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/install_in_colab.html)